# L1: NLP tasks with a simple interface 🗞️

HF API 키와 관련 Python 라이브러리를 로드합니다.

In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [2]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None, ENDPOINT_URL="https://api-inference.huggingface.co/models/sshleifer/distilbart-cnn-12-6"): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

## 텍스트 요약 앱 구축

여기서는 `facebook/bart-large-cnn`에서 증류한 306M 파라미터 모델인 `sshleifer/distilbart-cnn-12-6`을 위해 [허깅페이스 추론 엔드포인트](https://huggingface.co/inference-endpoints)를 이용합니다. 

### 로컬에서 실행하는 방법
API가 아닌 로컬에서 실행하는 경우 코드는 매우 유사하게 보일 것입니다. 나머지 과정의 모든 모델도 마찬가지입니다. 

[파이프라인](https://huggingface.co/docs/transformers/main_classes/pipelines) 문서 페이지 참고

```py
from transformers import pipeline

get_completion = pipeline("summarization", model="shleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
```

In [3]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

[{'summary_text': ' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It is the tallest structure in Paris and the second tallest free-standing structure in France after the Millau Viaduct . It was the first structure in the world to reach a height of 300 metres .'}]

### Gradio 시작하기 `gr.Interface` 

#### 로컬에서 실행하는 방법
로컬에서 실행하는 경우 코드가 매우 유사하게 보일 것입니다.  실행 메서드에서 모든 매개 변수를 제거하기만 하면 됩니다.

```py
demo.launch()
```

In [6]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


`demo.launch(share=True)`을 추가하여 팀이나 친구들과 공유할 공개 링크를 만들 수 있습니다.

In [7]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True)

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a8dbf1733d2815f06e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 네임드 엔티티 인식 앱 구축하기

여기서는 NET 타스크를 위해 미세조정된 108M 파라미터 BERT 모델인 `dslim/bert-base-NER`을 위해 [허깅페이스 추론 엔드포인트](https://huggingface.co/inference-endpoints)를 이용합니다. 

#### 로컬에서 실행하는 방법

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [8]:
API_URL = "https://api-inference.huggingface.co/models/dslim/bert-base-NER"

text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)

[{'entity_group': 'PER',
  'score': 0.9990624785423279,
  'word': 'Andrew',
  'start': 11,
  'end': 17},
 {'entity_group': 'ORG',
  'score': 0.896050214767456,
  'word': 'DeepLearningAI',
  'start': 32,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.999692440032959,
  'word': 'California',
  'start': 61,
  'end': 71}]

In [9]:
def fix_tokens(tokens):
    for token in tokens:
        if token['entity_group']:
            token['entity'] = token['entity_group']

    return tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    fixed_tokens = fix_tokens(output)
    # print(fixed_tokens)
    return {"text": input, "entities": fixed_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    # outputs=[gr.Textbox(label="Text with entities")],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch(share=True)

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://16c9b8cd4e99c78555.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 토큰 병합을 위한 도우미 기능 추가

In [10]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if token['entity_group']:
            token['entity'] = token['entity_group']

    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch(share=True)

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3ae8966da1c4b9b27a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
